<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [ ]:
%pip install numpy -q

In [ ]:
import functools  # For lru_cache memoization
import numpy as np
import heapq  # for priority queue
import copy  # for deepcopy of lists

In [ ]:
GOAL_STATE_8 = [
    [0, 1, 2],
    [
        3,
        4,
        5,
    ],
    [6, 7, 8],
]  # goal state for 8-puzzle - autoformatter may break formatting

GOAL_STATE_8_TUPLE = tuple(
    tuple(row) for row in GOAL_STATE_8
)  # Tuple version of goal state for set comparison

In [ ]:
class PuzzleNode:
    """
    PuzzleNode: for A* search of n^2-1 puzzle.
    Represents a state in search space. Holds puzzle config and A* info (parent, cost, heuristic).
    """

    def __init__(self, state, parent=None, cost=0, h=0):
        """
        Constructor for PuzzleNode. Initialize node attributes.
        Heuristic calculation is now decoupled and 'h' value is passed directly.
        :param state: puzzle state (list of lists)
        :param parent: parent PuzzleNode (for path reconstruction)
        :param cost: cost from start (g value)
        :param h: heuristic value (h value) - pre-calculated outside
        """
        self.state = state  # current puzzle state
        self.parent = parent  #  node from which current state is reached
        self.cost = cost  #  g(n), cost from start to current node
        self.h = h  # h(n), heuristic value - now passed directly
        self.f_value = cost + self.h  # f(n) = g(n) + h(n), total estimated cost

    def __str__(self):
        """
        String representation of puzzle state for printing. Visualize puzzle state.
        :return: string representing puzzle state in grid format
        """
        n = len(self.state)  # puzzle dimension
        output = ""
        for i in range(n):
            for j in range(n):
                output += str(self.state[i][j]) + " "  # add tile number and space
            output += "\n"  # new line for grid format
        return output

    def __eq__(self, other):
        """
        Checks if two PuzzleNodes are equal based on state. Compare states for node equality.
        :param other: the other PuzzleNode to compare
        :return: True if states are equal, False otherwise
        """
        return self.state == other.state  # equality based on tile config

    def __lt__(self, other):
        """
        Less than comparison for priority queue based on f_value. Prioritize nodes in queue.
        :param other: the other PuzzleNode to compare
        :return: True if self's f_value < other's, False otherwise
        """
        return self.f_value < other.f_value  # lower f_value means higher priority

In [ ]:
# TEST - Create an instance of PuzzleNode
initial_state = [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
initial_node = PuzzleNode(initial_state, cost=1, h=2)
print("Initial Puzzle Node:")
print(initial_node)

Initial Puzzle Node:
1 2 3 
4 5 0 
6 7 8 



<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves. ~~ugly~~

In [ ]:
import functools  # For lru_cache memoization
import numpy as np


# Memoization decorator using functools.lru_cache for efficiency
def memoize(func):
    """
    Memoization decorator using functools.lru_cache.
    Caches the results of the function for previously seen inputs to avoid redundant computations.
    """

    @functools.lru_cache(maxsize=None)  # 'maxsize=None' means unlimited cache size
    def memoized_func(*args, **kwargs):
        """Memoized version of the function."""
        return func(*args, **kwargs)

    return memoized_func


# Create a goal state based on the size of the puzzle
def create_goal_state(n):
    """
    Creates the goal state for an n x n puzzle.
    ... (docstring remains the same) ...
    """
    goal_state = [[0 for _ in range(n)] for _ in range(n)]
    tile_val = 0
    for i in range(n):
        for j in range(n):
            goal_state[i][j] = tile_val
            tile_val += 1
    return goal_state


# Get the goal positions of tiles
def goal_state_position(n):
    """
    Pre-calculates and stores the goal positions (row, column) for each tile value in the goal state.
    ... (docstring remains the same) ...
    """
    goal_state = create_goal_state(n)
    position = {}
    for i in range(n):
        for j in range(n):
            position[goal_state[i][j]] = (i, j)
    return position


# Misplaced tiles heuristic
@memoize  # Apply memoization decorator to cache results
def h1(state):
    """
    Calculates the Misplaced Tiles heuristic for a given puzzle state.
    ... (docstring remains the same) ...
    """
    state_tuple = tuple(
        tuple(row) for row in state
    )  # Convert input state to tuple of tuples for memoization
    goal_state = create_goal_state(len(state))
    misplaced_tiles = 0
    n = len(state)

    for i in range(n):
        for j in range(n):
            if state[i][j] != goal_state[i][j] and state[i][j] != 0:
                misplaced_tiles += 1
    return misplaced_tiles


# Manhattan distance heuristic
@memoize  # Apply memoization decorator to cache results
def h2(state):
    """
    Calculates the Manhattan Distance heuristic for a given puzzle state.
    ... (docstring remains the same) ...
    """
    state_tuple = tuple(
        tuple(row) for row in state
    )  # Convert input state to tuple of tuples for memoization
    manhattan_distance = 0
    goal_positions = goal_state_position(len(state))
    n = len(state)

    for i in range(n):
        for j in range(n):
            tile = state[i][j]
            if tile != 0:
                goal_row, goal_col = goal_positions[tile]
                manhattan_distance += abs(i - goal_row) + abs(j - goal_col)
    return manhattan_distance


# Linear Conflict heuristic (h3) - as discussed previously, for extension
@memoize
def h3(state):
    """
    Calculates the Linear Conflict heuristic, which dominates Manhattan Distance.
    ... (docstring remains the same) ...
    """
    state_tuple = tuple(
        tuple(row) for row in state
    )  # Convert input state to tuple of tuples for memoization
    h_man = h2(state)
    linear_conflicts = 0
    n = len(state)
    goal_state = create_goal_state(n)

    # Check for row conflicts
    for i in range(n):
        tiles_in_row = []
        for j in range(n):
            tile = state[i][j]
            if tile != 0 and goal_state[i].count(tile):
                tiles_in_row.append((tile, j))
        tiles_in_row.sort(key=lambda x: goal_state[i].index(x[0]))
        for k in range(len(tiles_in_row)):
            for l in range(k + 1, len(tiles_in_row)):
                if tiles_in_row[k][1] > tiles_in_row[l][1]:
                    linear_conflicts += 2

    # Check for column conflicts (similar logic as row conflicts)
    for j in range(n):
        tiles_in_col = []
        for i in range(n):
            tile = state[i][j]
            goal_col = [goal_state[row][j] for row in range(n)]
            if tile != 0 and goal_col.count(tile):
                tiles_in_col.append((tile, i))
        tiles_in_col.sort(
            key=lambda x: [goal_state[row][j] for row in range(n)].index(x[0])
        )
        for k in range(len(tiles_in_col)):
            for l in range(k + 1, len(tiles_in_col)):
                if tiles_in_col[k][1] > tiles_in_col[l][1]:
                    linear_conflicts += 2

    return h_man + linear_conflicts


# Heuristic list
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [ ]:
def get_blank_pos(state):
    """
    Finds the position (row, col) of the blank tile (0) in a given puzzle state.

    Args:
        state (list of lists): The puzzle state as a 2D list.

    Returns:
        tuple: A tuple (row, column) representing the position of the blank tile.
               Returns None if the blank tile (0) is not found (which should not happen in a valid puzzle state).
    """
    n = len(state)  # Get puzzle dimension
    for i in range(n):
        for j in range(n):
            if state[i][j] == 0:  # Check if current tile is the blank tile (0)
                return i, j  # Return row and column index of blank tile
    return None  # Should not reach here in a valid n-puzzle state


def get_successors(node, heuristic):
    """
    Generates successor PuzzleNodes for a given PuzzleNode by exploring possible moves of the blank tile.

    For each valid move (up, down, left, right), it creates a new PuzzleNode representing the resulting state,
    with updated cost and heuristic value.

    Args:
        node (PuzzleNode): The current PuzzleNode for which to generate successors.
        heuristic (function): The heuristic function to be used to evaluate successor states.

    Returns:
        list of PuzzleNode: A list of successor PuzzleNodes.
    """
    current_state = node.state  # Get the state from the current node
    blank_row, blank_col = get_blank_pos(
        current_state
    )  # Find the position of the blank tile
    n = len(current_state)  # Puzzle dimension
    successors = []  # Initialize list to store successor nodes
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Possible moves: Right, Left, Down, Up

    for dr, dc in moves:  # Iterate through possible moves
        new_row, new_col = (
            blank_row + dr,
            blank_col + dc,
        )  # Calculate new position of blank tile
        if (
            0 <= new_row < n and 0 <= new_col < n
        ):  # Check if the move is within the board boundaries
            new_state = [
                list(row) for row in current_state
            ]  # Create a deep copy of the state to avoid modifying the original
            # Swap blank tile with the tile in the new position to create the new state
            new_state[blank_row][blank_col], new_state[new_row][new_col] = (
                new_state[new_row][new_col],
                new_state[blank_row][blank_col],
            )
            h_value = heuristic(
                new_state
            )  # Calculate heuristic value for the new state
            # Create a new PuzzleNode for the successor state, linking it to the current node and updating cost and heuristic
            successor_node = PuzzleNode(
                state=new_state, parent=node, cost=node.cost + 1, h=h_value
            )
            successors.append(successor_node)  # Add the successor node to the list
    return successors


def is_valid_state(state):
    """
    Checks if the input state is a valid n-puzzle state.
    Validates that the state is a list of lists, square, and contains all numbers from 0 to n^2 - 1 exactly once.

    Args:
        state (list of lists): The puzzle state to validate.

    Returns:
        bool: True if the state is valid, False otherwise.
    """
    if not isinstance(state, list):  # Check if state is a list
        return False
    n = len(state)  # Get dimension (assuming square puzzle)
    if n < 2:  # Minimum puzzle dimension is 2x2
        return False
    for row in state:
        if (
            not isinstance(row, list) or len(row) != n
        ):  # Check if rows are lists and have correct length
            return False

    expected_numbers = set(range(n * n))  # Expected numbers in a valid n-puzzle
    actual_numbers = set()  # Set to collect numbers from the given state
    for i in range(n):
        for j in range(n):
            if not isinstance(state[i][j], int):  # Check if each element is an integer
                return False
            actual_numbers.add(state[i][j])  # Add number to the set of actual numbers

    if actual_numbers != expected_numbers:  # Compare actual numbers with expected set
        return False  # State is invalid if numbers are not as expected
    return True  # State is valid if all checks pass

In [ ]:
def get_blank_pos(state):
    """
    Finds the position (row, col) of the blank tile (0) in a given puzzle state.
    """
    n = len(state)
    for i in range(n):
        for j in range(n):
            if state[i][j] == 0:
                return i, j
    return None


def get_successors(node, heuristic):
    """
    Generates successor PuzzleNodes for a given PuzzleNode by exploring possible moves of the blank tile.
    """
    current_state = node.state
    blank_row, blank_col = get_blank_pos(current_state)
    n = len(current_state)
    successors = []
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    for dr, dc in moves:
        new_row, new_col = blank_row + dr, blank_col + dc
        if 0 <= new_row < n and 0 <= new_col < n:
            new_state = [list(row) for row in current_state]
            new_state[blank_row][blank_col], new_state[new_row][new_col] = (
                new_state[new_row][new_col],
                new_state[blank_row][blank_col],
            )
            h_value = heuristic(new_state)
            successor_node = PuzzleNode(
                state=new_state, parent=node, cost=node.cost + 1, h=h_value
            )
            successors.append(successor_node)
    return successors


def is_valid_state(state):
    """
    Checks if the input state is a valid n-puzzle state.
    """
    if not isinstance(state, list):
        return False
    n = len(state)
    if n < 2:
        return False
    for row in state:
        if not isinstance(row, list) or len(row) != n:
            return False

    expected_numbers = set(range(n * n))
    actual_numbers = set()
    for i in range(n):
        for j in range(n):
            if not isinstance(state[i][j], int):
                return False
            actual_numbers.add(state[i][j])

    if actual_numbers != expected_numbers:
        return False
    return True


def count_inversions(state):
    """
    Counts the number of inversions in a given puzzle state.
    An inversion is a pair of tiles (i, j) where i appears before j in the linearized state,
    but i > j. Blank tile (0) is not counted.

    Args:
        state (list of lists): The puzzle state as a 2D list.

    Returns:
        int: The number of inversions in the state.
    """
    n = len(state)  # Puzzle dimension
    inversions = 0  # Initialize inversion counter
    linear_state = []  # Linearized state to easily check order

    for i in range(n):
        for j in range(n):
            if state[i][j] != 0:  # Exclude blank tile from inversion count
                linear_state.append(
                    state[i][j]
                )  # Create a 1D representation of the puzzle state

    for i in range(len(linear_state)):
        for j in range(i + 1, len(linear_state)):
            if (
                linear_state[i] > linear_state[j]
            ):  # Check for inversion: tile at i > tile at j but i < j in sequence
                inversions += 1  # Increment inversion count if inversion is found
    return inversions


def is_solvable(state):
    """
    Determines if a given n-puzzle state is solvable.
    Uses the inversion count method to check for solvability based on puzzle dimension (n).

    Args:
        state (list of lists): The puzzle state as a 2D list.

    Returns:
        bool: True if the state is solvable, False otherwise.
    """
    n = len(state)  # Puzzle dimension
    inversions = count_inversions(state)  # Count inversions in the given state

    if n % 2 == 1:  # For odd-sized puzzles (like 8-puzzle)
        return inversions % 2 == 0  # Solvable if inversion count is even
    else:  # For even-sized puzzles (like 15-puzzle)
        blank_row_from_bottom = (
            n - get_blank_pos(state)[0]
        )  # Row of blank tile from bottom (1-indexed)
        if blank_row_from_bottom % 2 == 0:  # Blank tile on even row from bottom
            return inversions % 2 == 1  # Solvable if inversion count is odd
        else:  # Blank tile on odd row from bottom
            return inversions % 2 == 0  # Solvable if inversion count is even


# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2.
    Implements the A* search algorithm and includes a solvability check.
    """
    if not is_valid_state(state):
        return 0, 0, 0, None, -1  # Error code -1 for invalid state

    if not is_solvable(state):  # Check if the puzzle is solvable
        return 0, 0, 0, None, -2  # Error code -2 for unsolvable state

    initial_h_value = heuristic(state)
    initial_node = PuzzleNode(state, parent=None, cost=0, h=initial_h_value)
    frontier = [(initial_node.f_value, initial_node)]
    heapq.heapify(frontier)
    explored_set = set()
    max_frontier_size = 1
    nodes_expanded = 0

    goal_state_tuple = GOAL_STATE_8_TUPLE

    while frontier:
        f_val, current_node = heapq.heappop(frontier)
        current_state_tuple = tuple(tuple(row) for row in current_node.state)

        if current_state_tuple == goal_state_tuple:
            path = []
            node = current_node
            while node:
                path.append(node.state)
                node = node.parent
            return current_node.cost, nodes_expanded, max_frontier_size, path[::-1], 0

        if current_state_tuple in explored_set:
            continue

        explored_set.add(current_state_tuple)
        nodes_expanded += 1

        successors = get_successors(current_node, heuristic)
        for successor in successors:
            heapq.heappush(frontier, (successor.f_value, successor))
        max_frontier_size = max(max_frontier_size, len(frontier))

    return (
        0,
        0,
        max_frontier_size,
        None,
        0,
    )  # Should not reach here for solvable puzzles

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [ ]:
## Test for state not correctly defined

incorrect_state = [[0, 1, 2], [2, 3, 4], [5, 6, 7]]
_, _, _, _, err = solvePuzzle(incorrect_state, lambda state: 0)
assert err == -1

In [ ]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = (
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[7, 0, 8], [4, 6, 1], [5, 3, 2]],
    [[5, 7, 6], [2, 4, 3], [8, 1, 0]],
)

# Test the values returned by the heuristic functions
h_mt_vals = [7, 8, 7]
h_man_vals = [15, 17, 18]

for i in range(0, 3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert h_mt == h_mt_vals[i]
    assert h_man == h_man_vals[i]

TypeError: unhashable type: 'list'

In [ ]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[2, 3, 7], [1, 8, 4], [6, 5, 0]],
    [[2, 3, 7], [1, 8, 4], [6, 0, 5]],
    [[2, 3, 7], [1, 0, 4], [6, 8, 5]],
    [[2, 0, 7], [1, 3, 4], [6, 8, 5]],
    [[0, 2, 7], [1, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [0, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 0, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 4, 0], [6, 8, 5]],
    [[1, 2, 0], [3, 4, 7], [6, 8, 5]],
    [[1, 0, 2], [3, 4, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 0, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 0], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 5], [6, 8, 0]],
    [[1, 4, 2], [3, 7, 5], [6, 0, 8]],
    [[1, 4, 2], [3, 0, 5], [6, 7, 8]],
    [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]],
]

astar_steps = [17, 25, 28]
for i in range(0, 3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[0]
    )
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_mt == steps_man == astar_steps[i]
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert expansions_man < expansions_mt
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert opt_path_mt == opt_path_soln

In [ ]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [
    [1, 2, 6, 3],
    [0, 9, 5, 7],
    [4, 13, 10, 11],
    [8, 12, 14, 15],
]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[0]
)
steps_man, expansions_man, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[1]
)
# Test whether the number of optimal steps is correct and the same
assert steps_mt == steps_man == 9
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert expansions_mt >= expansions_man

IndexError: list index out of range

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [ ]:
## Puzzle solvability test

unsolvable_initial_state = [[7, 5, 6], [2, 4, 3], [8, 1, 0]]
_, _, _, _, err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert err == -2

AssertionError: 

In [ ]:
## Extra heuristic function test.
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0, 3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[2]
    )
    steps_man, expansions_man, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_new == steps_man == astar_steps[i]
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert dom > 0

In [ ]:
## Memoization test - will be carried out after submission

In [ ]:
import heapq


class AbstractNode:
    def __init__(self, state, parent=None, cost=0, heuristic_value=0):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic_value = heuristic_value
        self.f_value = cost + heuristic_value

    def __eq__(self, other):
        return self.state == other.state

    def __lt__(self, other):
        return self.f_value < other.f_value


def abstract_get_successors(node):
    state = node.state
    if state == "S":
        return [
            AbstractNode("A", parent=node, cost=node.cost + 1),
            AbstractNode("B", parent=node, cost=node.cost + 1),
        ]
    elif state == "A":
        return [AbstractNode("G", parent=node, cost=node.cost + 1)]
    elif state == "B":
        return [
            AbstractNode("G", parent=node, cost=node.cost + 2)
        ]  # Higher cost path from B to G
    return []


def abstract_heuristic(state):
    if state == "S":
        return 2  # Perfect heuristic for S - optimal path is 2 steps (S->A->G)
    elif state == "A":
        return 1  # Perfect heuristic for A - optimal path is 1 step (A->G)
    elif state == "B":
        return 2  # Heuristic for B - suboptimal path is 2 steps (B->G)
    elif state == "G":
        return 0  # Goal state, heuristic is 0
    return 0


def abstract_solve_puzzle(start_state, heuristic_func):
    initial_node = AbstractNode(
        start_state, heuristic_value=heuristic_func(start_state)
    )
    frontier = [(initial_node.f_value, initial_node)]
    heapq.heapify(frontier)
    explored_set = set()
    max_frontier_size = 1
    nodes_expanded = 0
    goal_state = "G"

    while frontier:
        f_val, current_node = heapq.heappop(frontier)
        if current_node.state == goal_state:
            path = []
            node = current_node
            while node:
                path.append(node.state)
                node = node.parent
            return current_node.cost, nodes_expanded, max_frontier_size, path[::-1], 0

        if current_node.state in explored_set:
            continue
        explored_set.add(current_node.state)
        nodes_expanded += 1

        successors = abstract_get_successors(current_node)
        for successor in successors:
            successor.heuristic_value = heuristic_func(successor.state)
            successor.f_value = successor.cost + successor.heuristic_value
            heapq.heappush(frontier, (successor.f_value, successor))
        max_frontier_size = max(max_frontier_size, len(frontier))
    return 0, 0, max_frontier_size, None, 0


# New Test Case:
start_state_test = "S"
steps_test, expansions_test, max_frontier_test, opt_path_test, err_test = (
    abstract_solve_puzzle(start_state_test, abstract_heuristic)
)

print(f"Abstract Test Case Results:")
print(f"Steps: {steps_test}")
print(f"Expansions: {expansions_test}")
print(f"Max Frontier Size: {max_frontier_test}")
print(f"Optimal Path: {opt_path_test}")
print(f"Error Code: {err_test}")

Abstract Test Case Results:
Steps: 2
Expansions: 2
Max Frontier Size: 2
Optimal Path: ['S', 'A', 'G']
Error Code: 0
